<a href="https://colab.research.google.com/github/woncoh1/END/blob/main/Assignment_Session_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Instructions  
Use this as a [reference](https://github.com/bentrevett/pytorch-sentiment-analysis/blob/master/2%20-%20Upgraded%20Sentiment%20Analysis.ipynb). 

Change this code in such a way that:

- it has **3** LSTM layers &
- it has used a **for loop** to do so in the **forward** function
  - see **2. Build the Model**, *1) define model structure*
- the dropout value used is **0.2**  
  - see **2. Build the Model**, *2) define model constants*
- trained on the text that is **reversed** (for example, "my name is Rohan"
 becomes "Rohan is name my")
  - see **1. Prepare Data**, *3) reverse training text data in-place*
  - also, `bidirectional=False`
- achieves **87% or more** accuracy  
  - After 7 epochs, validation accuracy = **88.36%** and testing accuracy = **86.68%**

- once done, share the **Github link** as well (after training on Google Colab, move the file to GitHub).  

#1. Prepare Data

In [1]:
# 1) setup preprocessing

import torch
from torchtext import data

SEED = 1234

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

TEXT = data.Field(tokenize = 'spacy', include_lengths = True)
LABEL = data.LabelField(dtype = torch.float)

In [2]:
# 2) split training and testing data

from torchtext import datasets

train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

aclImdb_v1.tar.gz:   0%|          | 0.00/84.1M [00:00<?, ?B/s]

downloading aclImdb_v1.tar.gz


aclImdb_v1.tar.gz: 100%|██████████| 84.1M/84.1M [00:03<00:00, 23.2MB/s]


In [3]:
# 3) reverse training text data in-place

for train_datum in train_data.examples:
    vars(train_datum).get('text').reverse()

In [4]:
# 4) split validation data from training data

import random

train_data, valid_data = train_data.split(random_state = random.seed(SEED))

In [5]:
# 5) build vocabulary using pre-trained embeddings

MAX_VOCAB_SIZE = 25_000

TEXT.build_vocab(train_data, 
                 max_size = MAX_VOCAB_SIZE, 
                 vectors = "glove.6B.100d", 
                 unk_init = torch.Tensor.normal_)

LABEL.build_vocab(train_data)

.vector_cache/glove.6B.zip: 862MB [06:28, 2.22MB/s]                           
100%|█████████▉| 398320/400000 [00:16<00:00, 24225.58it/s]

In [6]:
# 6) build data (iterators of training, validation and testing batches)

BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE,
    sort_within_batch = True,
    device = device) # send data to GPU (if available)

#2. Build the Model

In [7]:
# 1) define model (neural network) structure

import torch.nn as nn

class RNN(nn.Module):

    #parts list for building blocks
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, 
                 n_layers, bidirectional, dropout, pad_idx):
        
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim, 
                                      padding_idx = pad_idx)
        
        self.rnns = nn.ModuleList([nn.LSTM(embedding_dim, hidden_dim, 
                                           bidirectional=bidirectional)])
        for _ in range(n_layers - 1):
            self.rnns.append(nn.LSTM(hidden_dim, hidden_dim, 
                                     bidirectional=bidirectional))
        
        self.fc = nn.Linear(hidden_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
    
    #step-by-step manual for assembling building blocks    
    def forward(self, text, text_lengths):
        
        #text = [sent len, batch size]
        
        embedded = self.dropout(self.embedding(text))
        
        #embedded = [sent len, batch size, emb dim]
        
        #pack sequence
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths)
        
        #stack multiple (3) LSTM layers with dropouts
        x = packed_embedded
        for rnn in self.rnns:
            _, (x, _) = rnn(x)
            x = self.dropout(x)
        
        #x = last hidden states = [1, batch size, hid dim]
        hidden = x.squeeze(0)

        #hidden = [batch size, hid dim]
            
        return self.fc(hidden)

In [8]:
# 2) define model constants

INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 1
N_LAYERS = 3          
BIDIRECTIONAL = False #changed from True to False
DROPOUT = 0.2         #changed from 0.5 to 0.2
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

In [9]:
# 3) instantiate model

model = RNN(INPUT_DIM, 
            EMBEDDING_DIM, 
            HIDDEN_DIM, 
            OUTPUT_DIM, 
            N_LAYERS, 
            BIDIRECTIONAL, 
            DROPOUT, 
            PAD_IDX)

In [10]:
# 4) count number of parameters (weights)

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 3,919,721 trainable parameters


In [11]:
# 5) inspect pre-trained embeddings

pretrained_embeddings = TEXT.vocab.vectors

print(pretrained_embeddings.shape)

torch.Size([25002, 100])


In [12]:
# 6) copy pre-trained embeddings from vocabulary to model

model.embedding.weight.data.copy_(pretrained_embeddings)

tensor([[-0.1117, -0.4966,  0.1631,  ...,  1.2647, -0.2753, -0.1325],
        [-0.8555, -0.7208,  1.3755,  ...,  0.0825, -1.1314,  0.3997],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [-0.3970,  0.4024,  1.0612,  ..., -0.0136, -0.3363,  0.6442],
        [-0.5197,  1.0395,  0.2092,  ..., -0.8857, -0.2294,  0.1244],
        [ 0.0057, -0.0707, -0.0804,  ..., -0.3292, -0.0130,  0.0716]])

In [13]:
# 7) zero weights for <unk> and <pad> tokens

UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

print(model.embedding.weight.data)

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [-0.3970,  0.4024,  1.0612,  ..., -0.0136, -0.3363,  0.6442],
        [-0.5197,  1.0395,  0.2092,  ..., -0.8857, -0.2294,  0.1244],
        [ 0.0057, -0.0707, -0.0804,  ..., -0.3292, -0.0130,  0.0716]])


#3. Train the Model

In [14]:
# 1) build optimizer (Adam)

import torch.optim as optim

optimizer = optim.Adam(model.parameters())

In [15]:
# 2) build loss function (Sigmoid + Binary Cross-Entropy Loss in one layer)

criterion = nn.BCEWithLogitsLoss()

In [16]:
# 3) send model and loss to GPU (if available)

model = model.to(device)
criterion = criterion.to(device)

In [17]:
# 4) define how to calculate accuracy for training, validation and testing

def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, 
    i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [18]:
# 5) define how to train the model

def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        
        text, text_lengths = batch.text

        #runtime type is GPU, while model expects CPU tensor
        text_lengths = text_lengths.cpu()

        predictions = model(text, text_lengths).squeeze(1)
        
        loss = criterion(predictions, batch.label)
        
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [19]:
# 6) define how to evaluate the model

def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            text, text_lengths = batch.text
            
            #runtime type is GPU, while model expects CPU tensor
            text_lengths = text_lengths.cpu()

            predictions = model(text, text_lengths).squeeze(1)
            
            loss = criterion(predictions, batch.label)
            
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [20]:
# 7) define how to calculate time spent per epoch

import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [21]:
# 8) data, model, loss and optimizer all come together to actually train model

N_EPOCHS = 7

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut2-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

100%|█████████▉| 398320/400000 [00:30<00:00, 24225.58it/s]

Epoch: 01 | Epoch Time: 0m 9s
	Train Loss: 0.693 | Train Acc: 51.07%
	 Val. Loss: 0.692 |  Val. Acc: 55.01%
Epoch: 02 | Epoch Time: 0m 9s
	Train Loss: 0.678 | Train Acc: 56.43%
	 Val. Loss: 0.694 |  Val. Acc: 49.15%
Epoch: 03 | Epoch Time: 0m 9s
	Train Loss: 0.693 | Train Acc: 50.54%
	 Val. Loss: 0.688 |  Val. Acc: 55.68%
Epoch: 04 | Epoch Time: 0m 9s
	Train Loss: 0.591 | Train Acc: 68.11%
	 Val. Loss: 0.646 |  Val. Acc: 58.57%
Epoch: 05 | Epoch Time: 0m 9s
	Train Loss: 0.602 | Train Acc: 67.01%
	 Val. Loss: 0.437 |  Val. Acc: 80.23%
Epoch: 06 | Epoch Time: 0m 9s
	Train Loss: 0.335 | Train Acc: 87.05%
	 Val. Loss: 0.337 |  Val. Acc: 86.73%
Epoch: 07 | Epoch Time: 0m 9s
	Train Loss: 0.218 | Train Acc: 92.00%
	 Val. Loss: 0.318 |  Val. Acc: 88.36%


In [22]:
# 9) test model using testing dataset

model.load_state_dict(torch.load('tut2-model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.359 | Test Acc: 86.68%


#4. User Input

In [23]:
# 1) define how to test the model using arbitrary user input

import spacy
nlp = spacy.load('en')

def predict_sentiment(model, sentence):
    model.eval()
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)]
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    length = [len(indexed)]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(1)
    length_tensor = torch.LongTensor(length)
    prediction = torch.sigmoid(model(tensor, length_tensor))
    return prediction.item()

In [24]:
# 2) test model using negative sentence

predict_sentiment(model, "This film is terrible")

0.11755318194627762

In [25]:
# 3) test model using positive sentence

predict_sentiment(model, "This film is great")

0.5729390978813171